# WordNet graphs

In [1]:
import break_utils
from nltk.corpus import wordnet as wn
from nltk.corpus.reader.wordnet import Synset
from collections import defaultdict
from operator import itemgetter
import networkx as nx
import matplotlib.pyplot as plt

In [2]:
ss = wn.synsets("break", "v")

In [3]:
edges = []
node_dict = {}

for s in ss:
    n = s.name()
    node_dict[n] = s
    for paths in s.hypernym_paths():
        for p in paths:
            pname = p.name()
            if pname != n:
                node_dict[pname] = p
                edges.append((n, pname))

In [4]:
nodes = [(n, {"synset": n}) for n, s in node_dict.items()]

In [5]:
G = nx.Graph()

In [6]:
G.add_nodes_from(nodes)

In [7]:
G.add_edges_from(edges)

In [8]:
def get_common_hypernym_def(node_names):
    all_intersect = []
    for n1 in node_names:
        for n2 in node_names:
            all_intersect.append(node_dict[n1].common_hypernyms(node_dict[n2]))
    x = set(all_intersect[0])
    for y in all_intersect[1: ]:
        x &= set(y)    
    x = list(x)[0]
    return x.definition()

In [9]:
subgraphs = sorted(nx.connected_components(G), key=len, reverse=True)

print(len(subgraphs))

for i, c in enumerate(subgraphs):
    if i >= len(break_utils.color_types):
        color = "black"
    else:    
        color = break_utils.color_types[i]
    g = G.subgraph(c)
    
    title = get_common_hypernym_def(g)
    
    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(11, 5))
    nx.draw(
        g, 
        pos=nx.kamada_kawai_layout(g), 
        ax=ax, with_labels=True, 
        node_size=0, 
        edge_color=color, 
        node_color=color, 
        font_color=color)
    ax.set_title(title)
    plt.tight_layout()    
    plt.savefig('fig/graph{}.png'.format(i), dpi=300)
    plt.close()

29
